In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorboard as tb    

np.set_printoptions(precision=3, suppress=True)

In [5]:
# Load data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Convert to float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Flatten images to vectors of length 784 (28*28)
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])

# Normalize the pixel intensities to make sure their values are between 0 to 1
# by dividing them by 255
x_train, x_test = x_train / 255., x_test / 255.

# Train on only 50000 images and keep 10000 images as validation data.
x_validate = x_train[0:10000,:]
y_validate = y_train[0:10000]

x_train = x_train[10000:,:]
y_train = y_train[10000:]

In [6]:
# MNIST dataset parameters
num_classes = 10
num_features = 784

In [7]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [13]:
# Predicted probability for each class
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function  
@tf.function
def loss(x, y):
    y_true = tf.one_hot(y, depth=num_classes)
    y_pred = logistic_regression(x)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), axis=1))

# Accuracy metric
def accuracy(x, y):
    y_prob = logistic_regression(x)
    correct_prediction = tf.equal(tf.argmax(y_prob, 1), tf.cast(y, tf.int64))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).numpy()
    return accuracy

In [9]:
# Training parameters
learning_rate = 0.01
training_steps = 1000
display_step = 50
optimizer = tf.optimizers.SGD(learning_rate)

In [14]:
# Optimization process 
for i in range(training_steps):
    with tf.GradientTape() as tape:
        current_loss = loss(x_train, y_train)
    gradiants = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradiants, [W, b]))
    if i % display_step == 0:
        print("i: %i, loss: %f" % (i, current_loss))

i: 0, loss: 2.302583
i: 50, loss: 1.860419
i: 100, loss: 1.556958
i: 150, loss: 1.346290
i: 200, loss: 1.196297
i: 250, loss: 1.085809
i: 300, loss: 1.001628
i: 350, loss: 0.935540
i: 400, loss: 0.882309
i: 450, loss: 0.838496
i: 500, loss: 0.801770
i: 550, loss: 0.770505
i: 600, loss: 0.743533
i: 650, loss: 0.720001
i: 700, loss: 0.699264
i: 750, loss: 0.680835
i: 800, loss: 0.664329
i: 850, loss: 0.649449
i: 900, loss: 0.635953
i: 950, loss: 0.623647


In [15]:
accuracy(x_train, y_train)

0.86038

In [ ]:
# Tensorboard setup
logdir = 'tflogs'
writer = tf.summary.create_file_writer(logdir)

In [18]:
# Optimization process with tensorboard
with writer.as_default():
    for i in range(training_steps):
        with tf.GradientTape() as tape:
            current_loss = loss(x_train,y_train)
        gradients = tape.gradient(current_loss, [W, b])
        optimizer.apply_gradients(zip(gradients, [W ,b]))
        # calculate the validation accuracy
        val_acc = accuracy(x_validate,y_validate)
        # write the value to tensorboard
        tf.summary.scalar('val_acc', val_acc, step=i)

In [19]:
# run tensorboard in colab or jupyter notebook
%tensorboard --logdir tflogs

UsageError: Line magic function `%tensorboard` not found.
